In [ ]:
from transformers import ClapModel, AutoProcessor
import torch
import librosa
import os
import pandas as pd
import numpy as np
from scipy.stats import pearsonr # For Pearson correlation
from sklearn.metrics import r2_score # For R-squared scores

In [ ]:
#CLAPmodel = "laion/larger_clap_general"
#CLAPmodel = "laion/larger_clap_music_and_speech"
CLAPmodel = "laion/larger_clap_music"
#CLAPmodel = "laion/clap-htsat-fused"
#CLAPmodel = "laion/clap-htsat-unfused"

In [ ]:
# Load CLAP model + processor
model = ClapModel.from_pretrained(CLAPmodel)
processor = AutoProcessor.from_pretrained(CLAPmodel)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/776M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/776M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
print('total number of CLAP\'s parameters:', sum(p.numel() for p in model.parameters()))

total number of CLAP's parameters: 193913882


In [ ]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('CLAP model size: {:.3f}MB'.format(size_all_mb))

CLAP model size: 740.294MB


In [ ]:
#This section is to convert the uploaded zip files into folders
import zipfile

# Define the paths to your zip files
zip_files = ["Exp1.zip", "Exp2.zip", "Analysis.zip"]

# Define the directory where you want to extract the files
extract_dir = "/content/" # You can change this if you want to extract elsewhere

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract each zip file
for zip_file in zip_files:
    if os.path.exists(zip_file):
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted {zip_file} to {extract_dir}")
    else:
        print(f"Erro r: {zip_file} not found.")

Extracted Exp1.zip to /content/
Extracted Exp2.zip to /content/
Extracted Analysis.zip to /content/


# Process audio

In [ ]:
audio_stimuli = []
stimuli_path = "/content/Exp1/Stimuli/"

for file in sorted(os.listdir(stimuli_path)):
    if file.endswith(".wav"):
        wav_path = os.path.join(stimuli_path, file)
        audio, sample_rate = librosa.load(wav_path, sr=48000)
        audio_stimuli.append(audio)

In [ ]:
inputs = processor(audios=audio_stimuli, return_tensors="pt", padding=True, sampling_rate=48000)
audio_embeddings = model.get_audio_features(**inputs)

In [ ]:
print(audio_embeddings.shape)

torch.Size([59, 512])


# Process text

In [ ]:
discrete_tags = ["happiness", "sadness", "anger", "tenderness", "fear"]

discrete_captions_perceived = ["I perceive this sound as " + tag for tag in discrete_tags]
print(discrete_captions_perceived)
discrete_captions_induced = ["This sound makes me feel " + tag for tag in discrete_tags]
print(discrete_captions_induced)

dimensional_tags = ["positive", "relaxed", "awake"]

dimensional_captions_perceived = ["I perceive this sound as " + tag for tag in dimensional_tags]
print(dimensional_captions_perceived)
dimensional_captions_induced = ["This sound makes me feel " + tag for tag in dimensional_tags]
print(dimensional_captions_induced)

['I perceive this sound as happiness', 'I perceive this sound as sadness', 'I perceive this sound as anger', 'I perceive this sound as tenderness', 'I perceive this sound as fear']
['This sound makes me feel happiness', 'This sound makes me feel sadness', 'This sound makes me feel anger', 'This sound makes me feel tenderness', 'This sound makes me feel fear']
['I perceive this sound as positive', 'I perceive this sound as relaxed', 'I perceive this sound as awake']
['This sound makes me feel positive', 'This sound makes me feel relaxed', 'This sound makes me feel awake']


In [ ]:
all_tags = discrete_captions_perceived + discrete_captions_induced + dimensional_captions_perceived + dimensional_captions_induced

#For Discrete Induced (IDisc)
Itag_inputs = processor(text=discrete_captions_induced, return_tensors="pt", padding=True)
Itag_embeds = model.get_text_features(**Itag_inputs)

#For Discrete Perceived (PDisc)
Ptag_inputs = processor(text=discrete_captions_perceived, return_tensors="pt", padding=True)
Ptag_embeds = model.get_text_features(**Ptag_inputs)

#For all of Discrete
tag_embeds = (Itag_embeds + Ptag_embeds)/2

# Generate outputs

In [ ]:
#For Discrete Induced (IDisc)
print("IDisc")
Isims = torch.matmul(audio_embeddings, Itag_embeds.T)
print(Isims.shape)

#For Discrete Perceived (PDisc)
print("PDisc")
Psims = torch.matmul(audio_embeddings, Ptag_embeds.T)
print(Psims.shape)

print("Disc")
#For all of Discrete
sims = torch.matmul(audio_embeddings, tag_embeds.T)
print(sims.shape)

IDisc
torch.Size([59, 5])
PDisc
torch.Size([59, 5])
Disc
torch.Size([59, 5])


## Load csv files and extract related columns

In [ ]:
def GetData(path, sims):
  IDim_path = path
  IDim_responses = []

  all_dfs = []

  for file in os.listdir(IDim_path):
      if file.endswith(".csv"):
          df = pd.read_csv(os.path.join(IDim_path, file))
          df.columns = df.columns.str.strip()
          required_columns = ["happiness", "sadness", "anger", "tenderness", "fear"]
          if all(col in df.columns for col in required_columns):
              all_dfs.append(df)

  mean_happiness_vector = []
  mean_sadness_vector = []
  mean_anger_vector = []
  mean_tenderness_vector = []
  mean_fear_vector = []

  num_rows = all_dfs[0].shape[0]

  for i in range(num_rows):
      current_row_happiness = []
      current_row_sadness = []
      current_row_anger = []
      current_row_tenderness = []
      current_row_fear = []

      # For the current row index 'i', collect values from all DataFrames
      for df in all_dfs:
          current_row_happiness.append(df.iloc[i]['happiness'])
          current_row_sadness.append(df.iloc[i]['sadness'])
          current_row_anger.append(df.iloc[i]['anger'])
          current_row_tenderness.append(df.iloc[i]['tenderness'])
          current_row_fear.append(df.iloc[i]['fear'])

      # Calculate the mean for the current row across all files, for each column
      mean_happiness_vector.append(np.mean(current_row_happiness))
      mean_sadness_vector.append(np.mean(current_row_sadness))
      mean_anger_vector.append(np.mean(current_row_anger))
      mean_tenderness_vector.append(np.mean(current_row_tenderness))
      mean_fear_vector.append(np.mean(current_row_fear))

  IDim_responses = {
      'happiness_vector': mean_happiness_vector,
      'sadness_vector': mean_sadness_vector,
      'anger_vector': mean_anger_vector,
      'tenderness_vector': mean_tenderness_vector,
      'fear_vector': mean_fear_vector
  }

  print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
  print(f"Length of happiness_vector: {len(IDim_responses['happiness_vector'])}")
  print(f"Length of sadness_vector: {len(IDim_responses['sadness_vector'])}")
  print(f"Length of anger_vector: {len(IDim_responses['anger_vector'])}")
  print(f"Length of tiredness_vector: {len(IDim_responses['tenderness_vector'])}")
  print(f"Length of fear_vector: {len(IDim_responses['fear_vector'])}")

  # Find the min and max values in the current sims tensor
  old_min = sims.min()
  old_max = sims.max()

  # Define the new desired range
  new_min = 1.0
  new_max = 9.0

  # Apply the min-max scaling formula
  scaled_sims = ((sims - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

  print(f"Scaled sims shape: {scaled_sims.shape}")
  print(f"Scaled sims (first 5 rows):\n{scaled_sims[:10]}\n")
  print(f"Scaled sims min value: {scaled_sims.min():.4f}")
  print(f"Scaled sims max value: {scaled_sims.max():.4f}\n")

  human_ratings_tensor = torch.tensor([
      IDim_responses['happiness_vector'],
      IDim_responses['sadness_vector'],
      IDim_responses['anger_vector'],
      IDim_responses['tenderness_vector'],
      IDim_responses['fear_vector']
  ], dtype=torch.float32).T # Transpose to get shape (59, 3)

  print(f"Human ratings tensor shape: {human_ratings_tensor.shape}")
  print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor[:10]}\n")

  return scaled_sims, human_ratings_tensor

Iscaled_sims, Ihuman_ratings_tensor = GetData('/content/Exp1/Data/IDisc', Isims)
Pscaled_sims, Phuman_ratings_tensor = GetData('/content/Exp1/Data/PDisc', Psims)


--- Mean Vectors (Mean across CSVs for each row position) ---
Length of happiness_vector: 59
Length of sadness_vector: 59
Length of anger_vector: 59
Length of tiredness_vector: 59
Length of fear_vector: 59
Scaled sims shape: torch.Size([59, 5])
Scaled sims (first 5 rows):
tensor([[6.6468, 7.1930, 7.2343, 7.1569, 6.6206],
        [4.9902, 5.4831, 5.5595, 5.5152, 4.9526],
        [3.1643, 3.6584, 3.7633, 3.6901, 3.1061],
        [2.7430, 3.2034, 3.2678, 3.1855, 2.7183],
        [7.3257, 7.8584, 7.9318, 7.8513, 7.3208],
        [7.8057, 8.3000, 8.4567, 8.3852, 7.8269],
        [2.3419, 2.8381, 3.0018, 2.9091, 2.3334],
        [4.2095, 4.6734, 4.7742, 4.6939, 4.1943],
        [5.7331, 6.2517, 6.3309, 6.2626, 5.6885],
        [4.5250, 4.9970, 5.0665, 4.9898, 4.4868]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 5])
Human ratings tensor (first 5 rows):
tensor([[1.8202, 3.3418, 2.7948, 1.9789, 4.1430],
  

In [ ]:
#Extracting data from aggregate_data.csv to get answers for purely Discrete
df = pd.read_csv('/content/Exp1/Data/aggregate_data.csv')

# Sort by StimNo to match audio alignment
df = df.sort_values("StimNo").reset_index(drop=True)

# Extract the mean responses
Dim_responses = {
    'happiness_vector': df['happiness'].tolist(),
    'sadness_vector': df['sadness'].tolist(),
    'anger_vector': df['anger'].tolist(),
    'tenderness_vector': df['tenderness'].tolist(),
    'fear_vector': df['fear'].tolist()
}

print("\n--- Mean Vectors (from aggregate_data.csv) ---")
print(f"Length of happiness_vector: {len(Dim_responses['happiness_vector'])}")
print(f"Length of sadness_vector: {len(Dim_responses['sadness_vector'])}")
print(f"Length of anger_vector: {len(Dim_responses['anger_vector'])}")
print(f"Length of tenderness_vector_vector: {len(Dim_responses['tenderness_vector'])}")
print(f"Length of fear_vector: {len(Dim_responses['fear_vector'])}")

# Find the min and max values in the current sims tensor
old_min = sims.min()
old_max = sims.max()

# Define the new desired range
new_min = 1.0
new_max = 9.0

# Apply the min-max scaling formula
scaled_sims = ((sims - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims[:10]}\n")
print(f"Scaled sims min value: {scaled_sims.min():.4f}")
print(f"Scaled sims max value: {scaled_sims.max():.4f}\n")

human_ratings_tensor = torch.tensor([
    Dim_responses['happiness_vector'],
    Dim_responses['sadness_vector'],
    Dim_responses['anger_vector'],
    Dim_responses['tenderness_vector'],
    Dim_responses['fear_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 5)

print(f"Human ratings tensor shape: {human_ratings_tensor.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor[:10]}\n")


--- Mean Vectors (from aggregate_data.csv) ---
Length of happiness_vector: 59
Length of sadness_vector: 59
Length of anger_vector: 59
Length of tenderness_vector_vector: 59
Length of fear_vector: 59
Scaled sims shape: torch.Size([59, 5])
Scaled sims (first 5 rows):
tensor([[6.6454, 7.2107, 7.2424, 7.2004, 6.6352],
        [5.0047, 5.5149, 5.5834, 5.5722, 4.9828],
        [3.2012, 3.7140, 3.8099, 3.7660, 3.1584],
        [2.7521, 3.2310, 3.2881, 3.2392, 2.7468],
        [7.2779, 7.8310, 7.8945, 7.8549, 7.2906],
        [7.7519, 8.2666, 8.4137, 8.3846, 7.7913],
        [2.3148, 2.8314, 2.9865, 2.9297, 2.3248],
        [4.1802, 4.6644, 4.7593, 4.7175, 4.1882],
        [5.7568, 6.2951, 6.3640, 6.3277, 5.7277],
        [4.5070, 4.9983, 5.0616, 5.0219, 4.4901]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 5])
Human ratings tensor (first 5 rows):
tensor([[1.6792, 3.8763, 3.9660, 2.0065, 4.8892],
        [

# Evaluate

In [ ]:
def format4(x):
    return "{:.4f}".format(float(x))

def format2(x):
    return "{:.2f}".format(float(x))

def format_tuple(t):
    return "(" + ", ".join(format4(x) for x in t) + ")"

def DisplayStats(Title, scaled_sims, human_ratings_tensor):
  # Comparison Method 1: Mean Absolute Error (MAE)
  # MAE measures the average magnitude of the errors in a set of predictions, without considering their direction.
  mae = torch.mean(torch.abs(scaled_sims - human_ratings_tensor))


  # Comparison Method 2: Pearson Correlation Coefficient (per column)
  # Pearson correlation measures the linear relationship between two sets of data.
  # We'll calculate it for each of the  columns.
  correlation_happiness, _ = pearsonr(scaled_sims[:, 0].detach().numpy(), human_ratings_tensor[:, 0].numpy())
  correlation_sadness, _ = pearsonr(scaled_sims[:, 1].detach().numpy(), human_ratings_tensor[:, 1].numpy())
  correlation_anger, _ = pearsonr(scaled_sims[:, 2].detach().numpy(), human_ratings_tensor[:, 2].numpy())
  correlation_tenderness, _ = pearsonr(scaled_sims[:, 3].detach().numpy(), human_ratings_tensor[:, 3].numpy())
  correlation_fear, _ = pearsonr(scaled_sims[:, 4].detach().numpy(), human_ratings_tensor[:, 4].numpy())

  correlation_scores = [correlation_happiness, correlation_sadness, correlation_anger, correlation_tenderness, correlation_fear]
  # Calculate the average correlation
  average_correlation = np.mean(correlation_scores)


  # Comparison Method 3: Mean Absolute Percentage Error (MAPE)
  # MAPE measures the accuracy of a forecasting method in terms of percentage.
  # Formula: MAPE = (1/n) * sum(|(Actual - Forecast) / Actual|) * 100%

  # Calculate the absolute percentage error for each element
  # Since human responses are between 1 and 9, division by zero is not a concern.
  absolute_percentage_error = torch.abs((human_ratings_tensor - scaled_sims) / human_ratings_tensor) * 100
  # Calculate the mean of these percentage errors
  mape = torch.mean(absolute_percentage_error)


  # Comparison Method 4: Root Mean Squared Error (RMSE)
  # RMSE measures the square root of the average of the squared differences between predicted and actual values.
  # It gives a relatively high weight to large errors.
  # Formula: RMSE = sqrt(mean((Actual - Forecast)^2))
  # Calculate the squared differences
  squared_differences = (human_ratings_tensor - scaled_sims)**2
  # Calculate the mean of the squared differences (Mean Squared Error - MSE)
  mse = torch.mean(squared_differences)
  # Calculate the square root to get RMSE
  rmse = torch.sqrt(mse)

  # Comparison Method 5: R-squared values (Coefficient of Determination)
  # R² measures how well the predicted values approximate the actual values.
  # It represents the proportion of variance in the dependent variable
  # that is predictable from the independent variable(s).
  #
  # Formula: R² = 1 - (Sum of Squared Residuals / Total Sum of Squares)
  #
  # Interpretation:
  # - R² = 1.0: Perfect prediction
  # - R² = 0.0: Predictions are as good as the mean of the actual values
  # - R² < 0.0: Predictions are worse than just using the mean
  r2_happiness = r2_score(human_ratings_tensor[:, 0].numpy(), scaled_sims[:, 0].detach().numpy())
  r2_sadness = r2_score(human_ratings_tensor[:, 1].numpy(), scaled_sims[:, 1].detach().numpy())
  r2_anger = r2_score(human_ratings_tensor[:, 2].numpy(), scaled_sims[:, 2].detach().numpy())
  r2_tenderness = r2_score(human_ratings_tensor[:, 3].numpy(), scaled_sims[:, 3].detach().numpy())
  r2_fear = r2_score(human_ratings_tensor[:, 4].numpy(), scaled_sims[:, 4].detach().numpy())

  r2_scores = [r2_happiness, r2_sadness, r2_anger, r2_tenderness, r2_fear]
  # Calculate the average R²
  average_r2 = np.mean(r2_scores)

  Disc_rows = [
    ("MAE", format4(mae)),
    ("Pearson Correlation Coefficient (Happiness, Sadness, Anger, Tenderness, Fear)",
    format_tuple(correlation_scores)),
    ("Average Correlation", format4(average_correlation)),
    ("MAPE", f"{format2(mape)}%"),
    ("RMSE", format4(rmse)),
    ("R-Squared Scores (Happiness, Sadness, Anger, Tenderness, Fear)",
    format_tuple(r2_scores)),
    ("R-Squared Average", format4(average_r2)),
  ]
  Disc_df = pd.DataFrame(Disc_rows, columns=[Title, "Statistics"])
  # Display with column borders, no row index
  style = [{"selector": "td, th", "props": [("border", "1px solid gray")]}]
  display(Disc_df.style.hide(axis="index").set_table_styles(style))


DisplayStats("Discrete Induced (IDisc)", Iscaled_sims, Ihuman_ratings_tensor)
print("\n" + "-"*80 + "\n")
DisplayStats("Discrete Perceived (PDisc)", Pscaled_sims, Phuman_ratings_tensor)
print("\n" + "-"*80 + "\n")
DisplayStats("Discrete", scaled_sims, human_ratings_tensor)

Discrete Induced (IDisc),Statistics
MAE,3.3042
"Pearson Correlation Coefficient (Happiness, Sadness, Anger, Tenderness, Fear)","(0.4549, -0.4340, -0.3218, 0.3611, -0.1315)"
Average Correlation,-0.0143
MAPE,148.60%
RMSE,3.8723
"R-Squared Scores (Happiness, Sadness, Anger, Tenderness, Fear)","(-12.6893, -45.0724, -13.7550, -30.6328, -20.5642)"
R-Squared Average,-24.5427



--------------------------------------------------------------------------------



Discrete Perceived (PDisc),Statistics
MAE,2.9056
"Pearson Correlation Coefficient (Happiness, Sadness, Anger, Tenderness, Fear)","(0.3447, -0.4720, -0.2168, 0.3270, -0.1895)"
Average Correlation,-0.0413
MAPE,115.60%
RMSE,3.4905
"R-Squared Scores (Happiness, Sadness, Anger, Tenderness, Fear)","(-7.2101, -10.5530, -9.7404, -12.9908, -7.0498)"
R-Squared Average,-9.5088



--------------------------------------------------------------------------------



Discrete,Statistics
MAE,3.0889
"Pearson Correlation Coefficient (Happiness, Sadness, Anger, Tenderness, Fear)","(0.4013, -0.4715, -0.2859, 0.3453, -0.1689)"
Average Correlation,-0.0360
MAPE,128.91%
RMSE,3.6680
"R-Squared Scores (Happiness, Sadness, Anger, Tenderness, Fear)","(-9.7988, -21.5312, -13.2474, -20.0062, -12.1004)"
R-Squared Average,-15.3368
